<a href="https://colab.research.google.com/github/sanjayathreya/Project/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sanjayathreya/Project.git
!mv /content/Project /content/CHET

Cloning into 'CHET'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 34 (delta 0), reused 2 (delta 0), pack-reused 29
Unpacking objects: 100% (34/34), 3.07 MiB | 4.60 MiB/s, done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir /content/CHET/data
!mkdir /content/CHET/data/mimic3/
!mkdir /content/CHET/data/mimic4/

In [4]:
!cp -a /content/drive/MyDrive/CHET/data/mimic3/raw/ /content/CHET/data/mimic3/
!cp -a /content/drive/MyDrive/CHET/data/mimic4/raw/ /content/CHET/data/mimic4/

In [5]:
!pip install pyhealth
!pip install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 15.9 MB/s eta 0:00:00


In [6]:
%cd /content/CHET

/content/CHET


In [7]:
%pwd

'/content/CHET'

In [14]:
import pandas as pd
from pyhealth.medcode import CrossMap
from pyhealth.datasets import MIMIC4Dataset,MIMIC3Dataset
from pyhealth.medcode import InnerMap
import os
import pickle
icd9cm = InnerMap.load("ICD9CM")

In [9]:
!cp /content/CHET/ICD10CM_to_ICD9CM.csv /root/.cache/pyhealth/medcode

In [10]:
mimic3_ds = MIMIC3Dataset(
    # Argument 1: It specifies the data folder root.
    root="data/mimic3/raw",

    # Argument 2: The users need to input a list of raw table names (e.g., DIAGNOSES_ICD.csv, PROCEDURES_ICD.csv).
    tables=["DIAGNOSES_ICD"],

    # Argument 3: This argument input a dictionary (key is the source code
    # vocabulary and value is the target code vocabulary .
    # Default is empty dict, which means the original code will be used.
)

Mapping codes: 100%|██████████| 23692/23692 [00:00<00:00, 124555.13it/s]


In [11]:
from collections import OrderedDict
icd9cm = InnerMap.load("ICD9CM")
patient_admission = OrderedDict()
admission_codes = OrderedDict()

patient_dict = mimic3_ds.patients
del_pid = {}
for pid, values in patient_dict.items():
  patient = patient_dict[pid]
  visit_dict = patient.visits
  if(len(visit_dict) >=2):
    admissions = []
    for visit_key, visit_values in visit_dict.items():
      diagnoses = visit_values.get_code_list(table="DIAGNOSES_ICD")
      diagnoses_std = [icd9cm.standardize(code) for code in diagnoses]
      admissions.append({'adm_id': visit_key, 'adm_time': visit_values.encounter_time})
      admission_codes[visit_key] = diagnoses_std
      if (len(diagnoses) == 0):
        del_pid[pid] = pid
    patient_admission[pid] = sorted(admissions, key=lambda admission: admission['adm_time'])

for pid in del_pid.keys():
  patient = patient_dict[pid]
  visit_dict = patient.visits
  del patient_admission[pid]
  for visit_key, visit_values in visit_dict.items():
    del admission_codes[visit_key]

In [12]:
patient_num = len(patient_admission)
max_admission_num = max([len(admissions) for admissions in patient_admission.values()])
avg_admission_num = sum([len(admissions) for admissions in patient_admission.values()]) / patient_num
max_visit_code_num = max([len(codes) for codes in admission_codes.values()])
avg_visit_code_num = sum([len(codes) for codes in admission_codes.values()]) / len(admission_codes)
print('patient num: %d' % patient_num)
print('max admission num: %d' % max_admission_num)
print('mean admission num: %.2f' % avg_admission_num)
print('max code num in an admission: %d' % max_visit_code_num)
print('mean code num in an admission: %.2f' % avg_visit_code_num)

patient num: 2169
max admission num: 23
mean admission num: 2.45
max code num in an admission: 39
mean code num in an admission: 10.70


In [16]:
%pwd

'/content/CHET'

In [15]:
data_path = 'data'
dataset = 'mimic3'  # mimic3, eicu, or mimic4
dataset_path = os.path.join(data_path,dataset)
parsed_path = os.path.join(dataset_path, 'parsed')
print('saving parsed data ...')
if not os.path.exists(parsed_path):
  os.makedirs(parsed_path)
pickle.dump(patient_admission, open(os.path.join(parsed_path, 'patient_admission.pkl'), 'wb'))
pickle.dump(admission_codes, open(os.path.join(parsed_path, 'admission_codes.pkl'), 'wb'))

saving parsed data ...


In [17]:
mimic4_ds = MIMIC4Dataset(
    # Argument 1: It specifies the data folder root.
    root="data/mimic4/raw",

    # Argument 2: The users need to input a list of raw table names (e.g., DIAGNOSES_ICD.csv, PROCEDURES_ICD.csv).
    tables=["diagnoses_icd"],

    # Argument 3: This argument input a dictionary (key is the source code
    # vocabulary and value is the target code vocabulary .
    # Default is empty dict, which means the original code will be used.
    code_mapping={"ICD10CM": "ICD9CM"},
)

Mapping codes: 100%|██████████| 180733/180733 [00:16<00:00, 10962.33it/s]


In [40]:
mimic4_ds.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC4Dataset
	- Number of patients: 180733
	- Number of visits: 431231
	- Number of visits per patient: 2.3860
	- Number of events per visit in DIAGNOSES_ICD: 0.0000



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC4Dataset\n\t- Number of patients: 180733\n\t- Number of visits: 431231\n\t- Number of visits per patient: 2.3860\n\t- Number of events per visit in DIAGNOSES_ICD: 0.0000\n'

In [38]:
patient4_dict = mimic4_ds.patients
print(list(patient4_dict.keys())[:10])

['10000032', '10000068', '10000084', '10000108', '10000117', '10000248', '10000280', '10000560', '10000635', '10000719']


In [39]:
patient_ = patient4_dict["10000032"]

In [41]:
from collections import OrderedDict
patient_admission = OrderedDict()
admission_codes = OrderedDict()

patient_dict = mimic4_ds.patients
del_pid = {}
for pid, values in patient_dict.items():
  patient = patient_dict[pid]
  visit_dict = patient.visits
  if(len(visit_dict) >=2):
    admissions = []
    for visit_key, visit_values in visit_dict.items():
      diagnoses = visit_values.get_code_list(table="diagnoses_icd")
      diagnoses_std = [icd9cm.standardize(code) for code in diagnoses]
      admissions.append({'adm_id': visit_key, 'adm_time': visit_values.encounter_time})
      admission_codes[visit_key] = diagnoses_std
      if (len(diagnoses) == 0):
        del_pid[pid] = pid
    patient_admission[pid] = sorted(admissions, key=lambda admission: admission['adm_time'])

for pid in del_pid.keys():
  patient = patient_dict[pid]
  visit_dict = patient.visits
  del patient_admission[pid]
  for visit_key, visit_values in visit_dict.items():
    del admission_codes[visit_key]

In [43]:
patient_num = len(patient_admission)
max_admission_num = max([len(admissions) for admissions in patient_admission.values()])
avg_admission_num = sum([len(admissions) for admissions in patient_admission.values()]) / patient_num
max_visit_code_num = max([len(codes) for codes in admission_codes.values()])
avg_visit_code_num = sum([len(codes) for codes in admission_codes.values()]) / len(admission_codes)
print('patient num: %d' % patient_num)
print('max admission num: %d' % max_admission_num)
print('mean admission num: %.2f' % avg_admission_num)
print('max code num in an admission: %d' % max_visit_code_num)
print('mean code num in an admission: %.2f' % avg_visit_code_num)

patient num: 55875
max admission num: 95
mean admission num: 3.69
max code num in an admission: 39
mean code num in an admission: 9.23


In [44]:
dataset = 'mimic4'  # mimic3, eicu, or mimic4
dataset_path = os.path.join(data_path,dataset)
parsed_path = os.path.join(dataset_path, 'parsed')


In [48]:
import numpy as np
seeds = [6669, 1000, 1050, 2052, 3000]
sample_num = 10000
for idx, seed in enumerate(seeds):
  np.random.seed(seed)
  keys = list(patient_admission.keys())
  selected_pids = np.random.choice(keys, sample_num, False)
  patient_admission_sample = {pid: patient_admission[pid] for pid in selected_pids}
  admission_codes_sample = dict()
  for admissions in patient_admission_sample.values():
      for admission in admissions:
          adm_id = admission['adm_id']
          admission_codes_sample[adm_id] = admission_codes[adm_id]
  print('saving parsed data ...')
  parsed_path_sample = os.path.join(parsed_path,str(idx))
  if not os.path.exists(parsed_path_sample):
    os.makedirs(parsed_path_sample)
  pickle.dump(patient_admission_sample, open(os.path.join(parsed_path_sample, 'patient_admission.pkl'), 'wb'))
  pickle.dump(admission_codes_sample, open(os.path.join(parsed_path_sample, 'admission_codes.pkl'), 'wb'))

saving parsed data ...
saving parsed data ...
saving parsed data ...
saving parsed data ...
saving parsed data ...


In [51]:
for idx, seed in enumerate(seeds):
  parsed_path_sample = os.path.join(parsed_path,str(idx))
  patient_admission = pickle.load(open(os.path.join(parsed_path_sample, 'patient_admission.pkl'), 'rb'))
  admission_codes = pickle.load(open(os.path.join(parsed_path_sample, 'admission_codes.pkl'), 'rb'))
  patient_num = len(patient_admission)
  max_admission_num = max([len(admissions) for admissions in patient_admission.values()])
  avg_admission_num = sum([len(admissions) for admissions in patient_admission.values()]) / patient_num
  max_visit_code_num = max([len(codes) for codes in admission_codes.values()])
  avg_visit_code_num = sum([len(codes) for codes in admission_codes.values()]) / len(admission_codes)
  print('********SAMPLE #: %d *************' % (idx+1))
  print('patient num: %d' % patient_num)
  print('patient num: %d' % patient_num)
  print('max admission num: %d' % max_admission_num)
  print('mean admission num: %.2f' % avg_admission_num)
  print('max code num in an admission: %d' % max_visit_code_num)
  print('mean code num in an admission: %.2f' % avg_visit_code_num)

********SAMPLE #: 1 *************
patient num: 10000
patient num: 10000
max admission num: 64
mean admission num: 3.71
max code num in an admission: 39
mean code num in an admission: 9.29
********SAMPLE #: 2 *************
patient num: 10000
patient num: 10000
max admission num: 77
mean admission num: 3.68
max code num in an admission: 39
mean code num in an admission: 9.07
********SAMPLE #: 3 *************
patient num: 10000
patient num: 10000
max admission num: 94
mean admission num: 3.69
max code num in an admission: 39
mean code num in an admission: 9.23
********SAMPLE #: 4 *************
patient num: 10000
patient num: 10000
max admission num: 71
mean admission num: 3.69
max code num in an admission: 39
mean code num in an admission: 9.32
********SAMPLE #: 5 *************
patient num: 10000
patient num: 10000
max admission num: 52
mean admission num: 3.68
max code num in an admission: 39
mean code num in an admission: 9.26


In [52]:
# !cp -a /content/CHET/data/mimic4/parsed/ /content/drive/MyDrive/CHET/data/mimic4/parsed/
# !cp -a /content/CHET/data/mimic4/parsed/ /content/drive/MyDrive/CHET/data/mimic4/parsed/